In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("pyspark_notebook1") \
    .master("local[*]") \
    .config("spark.executor.memory", "1g") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database", "MyVehiclesData") \
    .config("spark.mongodb.input.collection", "ProcessedVehiclesData") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/georgiamparakou/.ivy2/cache
The jars for the packages stored in: /Users/georgiamparakou/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e3f20b8a-5e89-49d0-9fcf-524c1dfd5662;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 100ms :: artifacts dl 4ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules       

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType, LongType

In [5]:
custom_schema1 = StructType([
    StructField("_id", StructType([StructField("oid", StringType(), nullable=True)]), nullable=True),
    StructField("link", StringType(), nullable=True),
    StructField("vcount", LongType(), nullable=True),
    StructField("vspeed", DoubleType(), nullable=True),
    StructField("Time", TimestampType(), nullable=True)
])

In [6]:
df1 = spark.read.format("com.mongodb.spark.sql.DefaultSource") \
            .option("uri", "mongodb://127.0.0.1:27017/MyVehiclesData.ProcessedVehiclesData") \
            .load()


In [7]:
df1.printSchema()

root
 |-- Time: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- link: string (nullable = true)
 |-- vcount: long (nullable = true)
 |-- vspeed: double (nullable = true)



In [8]:
from pyspark.sql.functions import col, to_timestamp

In [9]:
df1 = df1.withColumn("Τime", to_timestamp(col("time"), "yyyy-MM-dd HH:mm:ss"))

In [10]:
df1 = df1.select("_id.oid", "link", "vspeed", "vcount", "Time")

In [11]:
#df = df.dropDuplicates(["_id.oid", "link", "vspeed", "vcount", "Time"])

In [12]:
df1.show(20, truncate=False)

+------------------------+--------+------------------+------+-------------------+
|oid                     |link    |vspeed            |vcount|Time               |
+------------------------+--------+------------------+------+-------------------+
|667ec936e654e4743b5215e4|N1I1    |28.333333333333332|3     |2024-06-28 17:31:22|
|667ec93ce654e4743b5215ea|I1S1    |15.0              |1     |2024-06-28 17:31:42|
|667ec93ce654e4743b5215eb|N1I1    |28.75             |4     |2024-06-28 17:31:22|
|667ec941e654e4743b5215f0|N1I1    |28.75             |4     |2024-06-28 17:31:22|
|667ec941e654e4743b5215f1|I1S1    |22.5              |2     |2024-06-28 17:31:42|
|667ec947e654e4743b5215f7|N1I1    |28.75             |4     |2024-06-28 17:31:22|
|667ec947e654e4743b5215f6|I1S1    |25.0              |3     |2024-06-28 17:31:42|
|667ec94de654e4743b5215fc|I1S1    |26.25             |4     |2024-06-28 17:31:42|
|667ec94de654e4743b5215fd|trip_end|-1.0              |1     |2024-06-28 17:31:52|
|667ec94de654e47